### Pokemon Pokedex Data Analytics

Wanted some fun data analytics to start with

In [92]:
## importing libraries 
import numpy as np
import pandas as pd
from prettytable import PrettyTable

## Loading our data
## not setting # as default index because of repetitions
df = pd.read_csv('../datasets/Pokemon.csv')#, index_col = 0)
# df.shape
df.head()
# df.sample(5)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


## Step 1 - Exploring the data

In [64]:
## Collecting pertinent values into variables for easy reference
columns = list(df.columns) # Don't forget DataFrame index

df.describe()


,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,362.813750,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,208.343798,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,1.000000,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,184.750000,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,364.500000,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,539.250000,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,721.000000,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


In [57]:
## placating my inner child : Who's the strongest?
df[df.Total == df.Total.max()]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
163,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True
164,150,MewtwoMega Mewtwo Y,Psychic,NaN,780,106,150,70,194,120,140,1,True
426,384,RayquazaMega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True


In [145]:
## Looking into power of different types
values = df.groupby(['Type 1']).Total.agg(['mean','min', 'max'])
max_max = df.groupby(['Type 1']).Total.agg(['mean','min', 'max']).idxmax()
min_max =  df.groupby(['Type 1']).Total.agg(['mean','min', 'max']).idxmin()

## making table
x = PrettyTable()
x.field_names = ['Category', 'Pokemon Type', 'Value']
x.add_row(['Max of mean values', max_max[0], round(values.loc[max_max[0], 'mean'], 2)])
x.add_row(['Max of min values', max_max[1], round(values.loc[max_max[1], 'min'], 2)])
x.add_row(['Max of max values', max_max[2], round(values.loc[max_max[2], 'max'], 2)])
x.add_row(['Minimum of mean values', min_max[0], round(values.loc[min_max[0], 'mean'], 2)])
x.add_row(['Minimum of min values', min_max[1], round(values.loc[min_max[1], 'min'], 2)])
x.add_row(['Minimum of max values', min_max[2], round(values.loc[min_max[2], 'max'], 2)])

print(x)

+------------------------+--------------+--------+
|        Category        | Pokemon Type | Value  |
+------------------------+--------------+--------+
|   Max of mean values   |    Dragon    | 550.53 |
|   Max of min values    |    Dragon    |  300   |
|   Max of max values    |    Dragon    |  780   |
| Minimum of mean values |     Bug      | 378.93 |
| Minimum of min values  |    Grass     |  180   |
| Minimum of max values  |    Poison    |  535   |
+------------------------+--------------+--------+


In [76]:
df.groupby('Generation').aggregate({
    'Total' : [len,'mean','min','max'],
    'Attack' : ['max'],
    'Defense' : ['max']
})

Total                       Attack Defense
             len        mean  min  max    max     max
Generation                                           
1            166  426.813253  195  780    190     180
2            106  418.283019  180  700    185     230
3            160  436.225000  190  780    180     230
4            121  459.016529  194  720    170     168
5            165  434.987879  255  700    170     145
6             82  436.378049  200  700    160     184

In [62]:
## longest name vs shortest
print("Longest name vs shortest : {} - [{}] vs. {} - [{}]".format(df.Name.max(), len(df.Name.max()), min(df.Name), len(min(df.Name))))

Longest name vs shortest : Zygarde50% Forme - [16] vs. Abomasnow - [9]


# There seems to be a mistake
When first running this, we had shellos as part of the sample. 
Abomasnow has a length of 9, whilst Shellos had 7. Hence we didn't truly find the shortest name (& probably the longest neither)

In [5]:
## Potential candidate for shortest :
df[df.Name == 'Shellos']


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
469,422,Shellos,Water,NaN,325,76,48,48,57,62,34,4,False


In [16]:
## Confirming we found a mistake
# even the following; there's over 500 names shorter than length of 9
len(df[df['Name'].str.len()  < 9].Name.unique())

# and even with max; there's plenty of names longer than 16 characters
# df[df['Name'].str.len() > 16].Name.describe()

547

In [56]:
## sourced : https://stackoverflow.com/questions/21295334/find-length-of-longest-string-in-pandas-dataframe-column
df.Name.str.len() == df.Name.map(len).max()

## double checking
## print(len(df.iloc[7]) == len(df.iloc[124]))

0      False
1      False
2      False
3      False
4      False
       ...  
795    False
796    False
797    False
798    False
799    False
Name: Name, Length: 800, dtype: bool

In [18]:
## Getting column names
columns = list(df.columns)
# print(*columns, sep = ' - ')

## Loading specific items
df.iloc[[798,786,12,329,794], [1]]

,Name
798,HoopaHoopa Unbound
786,GourgeistLarge Size
12,BlastoiseMega Blastoise
329,MawileMega Mawile
794,Zygarde50% Forme


In [12]:
## Understanding objects
## 1 - type() (what is this thing?
## 2 - dir() (what can I do with it?)
## 3 - help() (tell me more)

# data - HoopaHoopa Unbound ; GourgeistLarge Size ; BlastoiseMega Blastoise ; MawileMega Mawile